In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


### DATA做一些清整、編碼 挑選要的特徵欄位

In [ ]:
pima = pd.read_csv('imdb.csv')
pima.info()
pima['sequal'].isna()
# 將空值(非續集)轉為0，有值得為1(是續集)
pima['sequal'] =pima['sequal'].isna().apply(lambda x: 0 if x==True else 1)
# 檢查轉換有無正確
#pima['sequal'][500:]
# 判斷有賺錢為1 沒有為0
pima['make_money'] = np.where(pima['revenue_transfer'] - pima['budget_transfer'] > 0, 1, 0)
pima[['revenue_transfer','budget_transfer','make_money']]
actor_cols = ['actor1', 'actor2', 'actor3', 'actor4', 'actor5', 'actor6']
actor_dummies = pd.get_dummies(pima[actor_cols].fillna(''), prefix=actor_cols)
# 將所有缺值補為0
pima=pima.fillna(value=0)
# 再度確認
#pima.info()
# 挑出要用的欄位放進X,y為有賺錢沒賺錢
test=pima[['avg_rating','sequal','Armistice Day (Observed)','Christmas Day (Observed)','Independence Day (Observed)','Juneteenth National Independence Day (Observed)',"New Year's Day (Observed)",'Veterans Day (Observed)',"Washington's Birthday",'actor1_count','actor2_count','actor3_count','actor4_count','actor5_count','actor6_count']]
y=pima['make_money']
X = pd.concat([test,actor_dummies],axis=1)

In [ ]:
df=pd.read_csv('df_final.csv',index_col=False)

In [ ]:
#df.columns.get_loc('Christmas Day_new') 確認假日在哪 因上面有警告訊息
X=df[df.columns[58:]]

In [ ]:
# 有缺值模型跑不動
df[df.columns[49:]].dropna()

In [ ]:
df1=df.copy()
df1['make_money'] = np.where(df1['revenue_transfer'] - df1['budget_transfer'] > 0, 1, 0)
#df=df.fillna(value=0)

In [ ]:
# drop na之後y對不上
df2 = pd.concat([X, y], axis=1, join='inner')

In [ ]:
y=df2['make_money']

### 模型測試

In [ ]:
# Only XG
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
# le = LabelEncoder()
# y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
regressor = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print('r2: {}'.format(r2))
print('mse: {}'.format(mse))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

### 多模型比較

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
dtc = DecisionTreeClassifier(max_depth=5, min_samples_split=20)
rfc = RandomForestClassifier(max_depth=5)
# svc = SVC(kernel='rbf', gamma=0.7, C=1)
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
knn = KNeighborsClassifier(n_neighbors=5)
gbc = GradientBoostingClassifier(n_estimators=100, max_depth=5, min_samples_split=20, min_samples_leaf=10)
lgr = LogisticRegression(max_iter=10000)
xgbst = xgb.XGBClassifier(n_estimators=100, max_depth=5)

models = [
    (dtc, 'Decision Tree'),
    (rfc, 'Random Forest'),
    # (svc, 'SVM'),
    (mlp, 'Neural Network'),
    (knn, 'KNN'),
    (gbc, 'Gradient Boosting'),
    (lgr, 'Logistic Regression'),
    (xgbst, 'XGBoost')
]

# Train and evaluate models
for model, model_name in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"{model_name}:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("Confusion Matrix:\n", cm)

# Define model names
model_names = [model_name for model, model_name in models]

# Plot bar chart of model performance
accuracy_list = [accuracy_score(y_test, model.predict(X_test)) for model, model_name in models]
precision_list = [precision_score(y_test, model.predict(X_test)) for model, model_name in models]
recall_list = [recall_score(y_test, model.predict(X_test)) for model, model_name in models]
f1_list = [f1_score(y_test, model.predict(X_test)) for model, model_name in models]

x = np.arange(len(model_names))
width = 0.2
fig, ax = plt.subplots(figsize=(10, 5))
rects1 = ax.bar(x - width*1.5, accuracy_list, width, label='Accuracy')
rects2 = ax.bar(x - width/2, precision_list, width, label='Precision')
rects3 = ax.bar(x + width/2, recall_list, width, label='Recall')
rects4 = ax.bar(x + width*1.5, f1_list, width, label='F1 Score')

ax.set_ylabel('Score')
ax.set_xticks(x)
ax.set_xticklabels(model_names)
ax.legend()

plt.show()



In [ ]:
# 各模型箱型圖比較
predicted_scores = []
for model, model_name in models:
    predicted_scores.append(model.predict_proba(X_test)[:, 1])

# Plot box plots of model performance
fig, ax = plt.subplots(figsize=(10, 5))
ax.boxplot(predicted_scores)
ax.set_xticklabels(model_names)
ax.set_ylabel('Predicted Probability')
ax.set_title('Comparison of Model Predicted Probabilities')
plt.show()

In [ ]:
# Train and evaluate models
results = []
names = []
for model, model_name in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"{model_name}:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("Confusion Matrix:\n", cm)
    
    results.append([accuracy, precision, recall, f1])
    names.append(model_name)
